<a href="https://colab.research.google.com/github/hws2002/Deep_Learning_with_Keras/blob/main/Chapter3/Chapter3_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
num_samples_per_class = 1000
negative_samples = np.random.multivariate_normal(
    mean = [0,3],
    cov = [[1, 0.5],[0.5, 1]],
    size = num_samples_per_class)

positive_samples = np.random.multivariate_normal(
    mean = [3,0],
    cov = [[1, 0.5],[0.5, 1]],
    size = num_samples_per_class)

inputs = np.vstack((negative_samples, positive_samples)).astype(np.float32)
targets = np.vstack( (np.ones((num_samples_per_class,1), dtype = 'float32'),
                       np.zeros((num_samples_per_class,1), dtype = 'float32'))
)

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow import keras

class SimpleDense(keras.layers.Layer): # 모든 케라스 층은 Layer 클래스를 상속함
  def __init__(self, units, activation = None):
    super().__init__()
    self.units = units
    self.activation = activation

  def build(self, input_shape): # build() 메서드에서 가중치를 생성함.
    input_dim = input_shape[-1]
    self.W = self.add_weight(shape = (input_dim, self.units),
                             initializer = "random_normal")
    # add_weight()는 가중치를 간편하게 만들 수 있는 메서드임
    # self.W = tf.Variable(tf.random.uniform(w_shape)) 와 같이 독립적으로 변수를 생성하고
    # 층의 속성으로 할당할 수도 있음.

    self.b = self.add_weight(shape=(self.units,),
                             initializer='zeros')
  def call(self, inputs): # call() 메서드에서 정방향 패스 계산을 정의함.
    y = tf.matmul(inputs, self.W) + self.b
    if self.activation is not None:
      y = self.activation(y)
    return y

위 클래스의 인스턴스를 생성하면 텐서플로 텐서를 입력으로 받는 함수처럼 사용할 수 있음.

In [ ]:
my_dense = SimpleDense(units = 32, activation = tf.nn.relu)
input_tensor = tf.ones(shape = (2,784))

In [ ]:
output_tensor = my_dense(input_tensor)
print(output_tensor.shape)

(2, 32)


## 자동 크기 추론 : 동적으로 층 만들기

In [ ]:
model = keras.Sequential(([
    SimpleDense(32, activation = "relu"),
    SimpleDense(64, activation = "relu"),
    SimpleDense(32, activation = "relu"),
    SimpleDense(10, activation = "softmax")
]))

# 컴파일 단계 : 학습 과정 설정

In [ ]:
model = keras.Sequential([ # 선형 분류기를 지정함
													keras.layers.Dense(1)
])

model.compile(optimizer = "rmsprop", # 옵티마이저이름을 지정함
              loss = "mean_squared_error", # 손실 이름을 평균 제곱 오차로 지정함
              metrics = ['accuracy']) # 측정 자료를 리스트로 지정함.

In [ ]:
model.compile(optimizer = keras.optimizers.RMSprop(),
              loss = keras.losses.MeanSquaredError(),
              metrics = [keras.metrics.BinaryAccuracy()])

In [ ]:
model.compile(optimizer= keras.optimizers.RMSprop(learning_rate = 1e-4),
              loss = my_custom_loss,
              metrics = [my_custom_metric_1,my_custom_metric_2]
)

In [ ]:
history = model.fit(
    inputs,
    targets,
    epochs = 5,
    batch_size = 128
)

# Validation data 만들기

In [ ]:
import numpy as np
model = keras.Sequential([
    keras.layers.Dense(1)
])
model.compile(optimizer = keras.optimizers.RMSprop(learning_rate = 0.1),
              loss = keras.losses.MeanSquaredError(),
              metrics = [keras.metrics.BinaryAccuracy()])

indices_permutation = np.random.permutation(len(inputs))
shuffled_inputs = inputs[indices_permutation]
shuffled_targets = targets[indices_permutation]

In [ ]:
num_validation_samples = int(0.3 * len(inputs))
val_inputs = shuffled_inputs[:num_validation_samples]
val_targets = shuffled_targets[:num_validation_samples]
training_inputs = shuffled_inputs[num_validation_samples:]
training_targets = shuffled_targets[num_validation_samples:]

In [ ]:
model.fit(
    training_inputs,
    training_targets,
    epochs = 5,
    batch_size = 16,
    validation_data = (val_inputs, val_targets) # 검증 데이터는 검증 손실과 측정 지표를 모니터링하는 데만 사용됨.
)

Epoch 1/5
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - binary_accuracy: 0.7602 - loss: 0.6925 - val_binary_accuracy: 0.9933 - val_loss: 0.0406
Epoch 2/5
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - binary_accuracy: 0.9457 - loss: 0.0804 - val_binary_accuracy: 0.9967 - val_loss: 0.0364
Epoch 3/5
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - binary_accuracy: 0.9635 - loss: 0.0694 - val_binary_accuracy: 0.9833 - val_loss: 0.0563
Epoch 4/5
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.9642 - loss: 0.0705 - val_binary_accuracy: 0.8867 - val_loss: 0.1689
Epoch 5/5
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.9467 - loss: 0.0858 - val_binary_accuracy: 0.8100 - val_loss: 0.2265


In [ ]:
# 훈련이 끝난 후 검증 손실과 측정 지표를 계산하고 싶다면 evaluate() 메서드를 사용할 수 있음
loss_and_metrics = model.evaluate(val_inputs, val_targets, batch_size = 128)
loss_and_metrics

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8125 - loss: 0.2256 


[0.2265048772096634, 0.8100000023841858]

# 추론 : 훈련한 모델 사용하기

In [ ]:
predictions = model(new_inputs)

In [ ]:
val_inputs

(600, 2)

In [ ]:
predictions = model.predict(val_inputs, batch_size = 128)
predictions[:20]

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


array([[ 1.1879563 ],
       [ 0.16433913],
       [ 1.5033181 ],
       [ 0.4457913 ],
       [ 0.5249623 ],
       [ 0.50062543],
       [ 0.26026815],
       [ 1.6183867 ],
       [ 0.3032552 ],
       [ 0.5307097 ],
       [ 0.53480417],
       [ 0.43891937],
       [ 1.665787  ],
       [ 0.5955257 ],
       [ 0.21836883],
       [ 0.398727  ],
       [ 0.16995838],
       [-0.03166747],
       [ 0.28241727],
       [ 0.82580006]], dtype=float32)